In [4]:
import torch
import torch.nn as nn

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPS = 1e-3*0.75


# Define a multi-layer neural network with variable number of layers
class MultiLayerNet(nn.Module):
    def __init__(self, layer_sizes):
        super(MultiLayerNet, self).__init__()
        layers = []
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(nn.ReLU())  # Adding ReLU activation between layers
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

def compute_gradients_central_difference(model, x, y):
    # Initialize gradients for manual computation
    layer_gradients_manual = [torch.zeros_like(param) for param in model.parameters()]

    # Perform central difference method for gradient approximation
    for layer_idx, layer in enumerate(model.layers):
        if isinstance(layer, nn.Linear):
            for i in range(layer.weight.shape[0]):
                for j in range(layer.weight.shape[1]):
                    # Perturb weights
                    layer.weight.data[i, j] += EPS
                    # Forward pass with positive perturbation
                    z_pos = model(x)
                    loss_pos = nn.functional.binary_cross_entropy_with_logits(z_pos, y)
                    
                    # Perturb weights back
                    layer.weight.data[i, j] -= 2 * EPS
                    # Forward pass with negative perturbation
                    z_neg = model(x)
                    loss_neg = nn.functional.binary_cross_entropy_with_logits(z_neg, y)
                    
                    # Central difference gradient approximation
                    layer_gradients_manual[layer_idx][i, j] = (loss_pos - loss_neg) / (2 * EPS)
                    
                    # Restore weights
                    layer.weight.data[i, j] += EPS

            # Compute gradients for bias
            for i in range(layer.bias.shape[0]):
                # Perturb bias
                layer.bias.data[i] += EPS
                # Forward pass with positive perturbation
                z_pos = model(x)
                loss_pos = nn.functional.binary_cross_entropy_with_logits(z_pos, y)
                
                # Perturb bias back
                layer.bias.data[i] -= 2 * EPS
                # Forward pass with negative perturbation
                z_neg = model(x)
                loss_neg = nn.functional.binary_cross_entropy_with_logits(z_neg, y)
                
                # Central difference gradient approximation for bias
                layer_gradients_manual[layer_idx+1][i] = (loss_pos - loss_neg) / (2 * EPS)
                
                # Restore bias
                layer.bias.data[i] += EPS

    return layer_gradients_manual

# Create an instance of the network with n layers
layer_sizes = [5, 10, 7, 3]  # Example: 4 layers (input size, hidden sizes, output size)
model = MultiLayerNet(layer_sizes).to(device)

# Input tensor (batch size 1, input size 5)
x = torch.randn(5, requires_grad=True, device=device)

# Expected output tensor (batch size 1, output size 3)
y = torch.randint(0, 2, (3,), device=device).float()

# Compute gradients using central difference (manual computation)
layer_gradients_manual = compute_gradients_central_difference(model, x, y)

# Compute gradients using backward method (automatic differentiation)
model.zero_grad()
z = model(x)
loss = nn.functional.binary_cross_entropy_with_logits(z, y)
loss.backward()

# Initialize gradients for automatic differentiation
layer_gradients_automatic = [param.grad.clone() for param in model.parameters()]
for l in layer_gradients_manual:
    print(l.shape)
# Compare gradients
for idx, (grad_manual, grad_automatic) in enumerate(zip(layer_gradients_manual, layer_gradients_automatic)):
    print(f"Comparison of gradients for layer {idx} parameters:")
    print("Manual Calculation:\n", grad_manual)
    print("Automatic Differentiation:\n", grad_automatic)
    print("--------------------------------------------------")

torch.Size([10, 5])
torch.Size([10])
torch.Size([7, 10])
torch.Size([7])
torch.Size([3, 7])
torch.Size([3])
Comparison of gradients for layer 0 parameters:
Manual Calculation:
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.4902e-03, -4.2915e-03,  1.8676e-03, -1.3113e-03, -1.7484e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.0697e-01,  1.0109e-01, -4.3909e-02,  3.1749e-02,  4.1286e-02],
        [ 5.4955e-02, -5.1975e-02,  2.2531e-02, -1.6371e-02, -2.1259e-02],
        [-2.7816e-04,  2.7816e-04, -7.9473e-05,  3.9736e-05,  7.9473e-05],
        [-2.3564e-02,  2.2292e-02, -9.6957e-03,  6.9936e-03,  9.0599e-03],
        [-2.7061e-02,  2.5590e-02, -1.1086e-02,  8.0665e-03,  1.0451e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-9.5367e-03,  9.0599e-03, -3.8942e-03,  2.8213e-03,  3.6558e-03]],
       device='cuda:0', grad_fn=<CopySlices>)
Automatic Differentiation: